In [1]:
import scipy.io as sio
import tensorflow as tf
import numpy as np

### 下載資料並進行整理

In [2]:
!wget https://github.com/toonnyy8/mhmc/raw/master/MultimediaProcessing/Data.mat

--2020-09-02 14:47:20--  https://github.com/toonnyy8/mhmc/raw/master/MultimediaProcessing/Data.mat
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/toonnyy8/mhmc/master/MultimediaProcessing/Data.mat [following]
--2020-09-02 14:47:20--  https://raw.githubusercontent.com/toonnyy8/mhmc/master/MultimediaProcessing/Data.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152437 (149K) [application/octet-stream]
Saving to: ‘Data.mat.1’

Data.mat.1          100%[===================>] 148.86K  --.-KB/s    in 0.04s   

2020-09-02 14:47:20 (3.68 MB/s) - ‘Data.mat.1’ saved [152437/152437]



In [3]:
data = sio.loadmat('Data.mat')

In [4]:
def to_np_array(data):
  nparr = []

  for _datas in data:
    _arr = []
    for _data in _datas:
      _arr.append(_data)
    nparr.append(_arr)

  return np.array(nparr)

In [5]:
channel1 = to_np_array(data['c1']).transpose((1,0,2,3))
channel2 = to_np_array(data['c2']).transpose((1,0,2,3))
e = 0.1
label = to_np_array(data['lable']).transpose((1,0,2,3))
label = np.maximum(label-e,0)
label = np.maximum(label,e/(label.shape[3]-1))

### 建立模型

In [6]:
def build_model(output):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(16,tf.keras.activations.swish),
    tf.keras.layers.Dense(output,tf.keras.activations.softmax),
  ])
  model.compile(tf.optimizers.Adam(),tf.losses.categorical_crossentropy,['accuracy'])
  return model

### Model 1
只使用 channel 1

In [7]:
models_1 = [] 
for i in range(5):
  model = build_model(5)
  x_train = np.concatenate([channel1[:(i)*3],channel1[(i+1)*3:]]).reshape((-1,channel1.shape[3]))
  y_train = np.concatenate([label[:(i)*3],label[(i+1)*3:]]).reshape((-1,label.shape[3]))
  model.fit(x_train, y_train, epochs=500)
  x_test = channel1[i*3:(i+1)*3].reshape((-1,channel1.shape[3]))
  y_test = label[i*3:(i+1)*3].reshape((-1,label.shape[3]))
  model.evaluate(x_test, y_test)
  models_1.append(model)

串流輸出內容已截斷至最後 5000 行。
Epoch 4/500
57/57 [==============================] - 0s 1ms/step - loss: 0.9034 - accuracy: 0.7711
Epoch 5/500
57/57 [==============================] - 0s 1ms/step - loss: 0.9014 - accuracy: 0.7739
Epoch 6/500
57/57 [==============================] - 0s 1ms/step - loss: 0.8895 - accuracy: 0.7800
Epoch 7/500
57/57 [==============================] - 0s 1ms/step - loss: 0.8922 - accuracy: 0.7661
Epoch 8/500
57/57 [==============================] - 0s 1ms/step - loss: 0.8749 - accuracy: 0.7806
Epoch 9/500
57/57 [==============================] - 0s 1ms/step - loss: 0.8680 - accuracy: 0.7844
Epoch 10/500
57/57 [==============================] - 0s 1ms/step - loss: 0.8692 - accuracy: 0.7850
Epoch 11/500
57/57 [==============================] - 0s 1ms/step - loss: 0.8595 - accuracy: 0.7917
Epoch 12/500
57/57 [==============================] - 0s 1ms/step - loss: 0.8514 - accuracy: 0.8000
Epoch 13/500
57/57 [==============================] - 0s 1ms/step - loss: 0.8638 - ac

### Model 2
只使用 channel 2

In [8]:
models_2 = [] 
for i in range(5):
  model = build_model(5)
  x_train = np.concatenate([channel2[:(i)*3],channel2[(i+1)*3:]]).reshape((-1,channel2.shape[3]))
  y_train = np.concatenate([label[:(i)*3],label[(i+1)*3:]]).reshape((-1,label.shape[3]))
  model.fit(x_train, y_train, epochs=500)
  x_test = channel2[i*3:(i+1)*3].reshape((-1,channel2.shape[3]))
  y_test = label[i*3:(i+1)*3].reshape((-1,label.shape[3]))
  model.evaluate(x_test, y_test)
  models_2.append(model)

串流輸出內容已截斷至最後 5000 行。
Epoch 4/500
57/57 [==============================] - 0s 1ms/step - loss: 1.1817 - accuracy: 0.6094
Epoch 5/500
57/57 [==============================] - 0s 1ms/step - loss: 1.1276 - accuracy: 0.6344
Epoch 6/500
57/57 [==============================] - 0s 1ms/step - loss: 1.0822 - accuracy: 0.6572
Epoch 7/500
57/57 [==============================] - 0s 2ms/step - loss: 1.0453 - accuracy: 0.6839
Epoch 8/500
57/57 [==============================] - 0s 1ms/step - loss: 1.0138 - accuracy: 0.7089
Epoch 9/500
57/57 [==============================] - 0s 1ms/step - loss: 0.9992 - accuracy: 0.6922
Epoch 10/500
57/57 [==============================] - 0s 1ms/step - loss: 0.9716 - accuracy: 0.7228
Epoch 11/500
57/57 [==============================] - 0s 1ms/step - loss: 0.9644 - accuracy: 0.7200
Epoch 12/500
57/57 [==============================] - 0s 1ms/step - loss: 0.9634 - accuracy: 0.7217
Epoch 13/500
57/57 [==============================] - 0s 1ms/step - loss: 0.9410 - ac

### Model 3
將 channel 1 與 channel 2 一起使用

In [9]:
models_3 = [] 
for i in range(5):
  connect_data = np.concatenate([channel1,channel2],axis=3)
  model = build_model(5)
  x_train = np.concatenate([connect_data[:(i)*3],connect_data[(i+1)*3:]]).reshape((-1,connect_data.shape[3]))
  y_train = np.concatenate([label[:(i)*3],label[(i+1)*3:]]).reshape((-1,label.shape[3]))
  model.fit(x_train, y_train, epochs=500)
  x_test = connect_data[i*3:(i+1)*3].reshape((-1,connect_data.shape[3]))
  y_test = label[i*3:(i+1)*3].reshape((-1,label.shape[3]))
  model.evaluate(x_test, y_test)
  models_3.append(model)

串流輸出內容已截斷至最後 5000 行。
Epoch 4/500
57/57 [==============================] - 0s 1ms/step - loss: 0.8642 - accuracy: 0.8050
Epoch 5/500
57/57 [==============================] - 0s 1ms/step - loss: 0.8144 - accuracy: 0.8378
Epoch 6/500
57/57 [==============================] - 0s 1ms/step - loss: 0.7900 - accuracy: 0.8494
Epoch 7/500
57/57 [==============================] - 0s 1ms/step - loss: 0.7736 - accuracy: 0.8606
Epoch 8/500
57/57 [==============================] - 0s 1ms/step - loss: 0.7484 - accuracy: 0.8761
Epoch 9/500
57/57 [==============================] - 0s 1ms/step - loss: 0.7379 - accuracy: 0.8856
Epoch 10/500
57/57 [==============================] - 0s 1ms/step - loss: 0.7297 - accuracy: 0.8906
Epoch 11/500
57/57 [==============================] - 0s 1ms/step - loss: 0.7202 - accuracy: 0.8856
Epoch 12/500
57/57 [==============================] - 0s 1ms/step - loss: 0.7094 - accuracy: 0.8922
Epoch 13/500
57/57 [==============================] - 0s 1ms/step - loss: 0.7109 - ac